In [ ]:
# !pip install ../input/kerasapplications/keras-team-keras-applications-3b180cb -f ./ --no-index
# !pip install ../input/efficientnet/efficientnet-1.1.0/ -f ./ --no-index

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data proc

import matplotlib.pyplot as plt
import seaborn as sns
import pydicom
# import gdcm
import os
import matplotlib.gridspec as gridspec
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import ZeroPadding2D,UpSampling2D,ThresholdedReLU,Conv2DTranspose, Cropping2D, DepthwiseConv2D,add, Activation, LeakyReLU, Dense, Conv2D, GlobalMaxPooling2D , MaxPooling2D, Flatten,Concatenate, Input, Dropout, BatchNormalization, GlobalAveragePooling2D, SeparableConv2D, AveragePooling2D
import cv2
from tensorflow.keras.optimizers import RMSprop, Adam, SGD 
from tensorflow.keras.metrics import TruePositives, FalsePositives, TrueNegatives, FalseNegatives, AUC, BinaryAccuracy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.activations import softsign
from keras.initializers import Constant
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold,StratifiedKFold,StratifiedShuffleSplit
import random
import keras.backend as K
import gc
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor,NearestNeighbors
from sklearn.linear_model import LinearRegression,Ridge, Lasso, LogisticRegression
from sklearn.ensemble import AdaBoostRegressor,GradientBoostingRegressor
from sklearn.linear_model import ElasticNet, BayesianRidge
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import OneHotEncoder
# import efficientnet.tfkeras as efn

In [ ]:
TRAIN = pd.read_csv("../input/osic-pulmonary-fibrosis-progression/train.csv")
TRAIN22 = pd.read_csv("../input/osic-pulmonary-fibrosis-progression/train.csv")
TEST =  pd.read_csv("../input/osic-pulmonary-fibrosis-progression/test.csv")
SUB =  pd.read_csv("../input/osic-pulmonary-fibrosis-progression/sample_submission.csv")
TRAIN_DIR = "../input/osic-pulmonary-fibrosis-progression/train/"
TEST_DIR = "../input/osic-pulmonary-fibrosis-progression/test/"
# TRAIN = TRAIN.loc[~TRAIN['Patient'].isin(["ID00011637202177653955184","ID00052637202186188008618"])]
TRAIN.reset_index(drop=True, inplace=True)
TEST.reset_index(drop=True, inplace=True)

BATCH = 15
SHAPE_RESIZE = 256
CUT = 10
COUNT_MODEL = 4
TRAIN["dir"] = TRAIN_DIR
TEST["dir"] = TEST_DIR#Todo
# TRAIN22["dir"] = TRAIN_DIR
# PACIENT_TEST = TRAIN["Patient"].unique().tolist()
# count = int(len(PACIENT_TEST)*80/100)
# PACIENT_train = random.sample(set(PACIENT_TEST), count)
# PACIENT_val = np.setdiff1d(PACIENT_TEST,PACIENT_train).tolist()
# TRAIN = TRAIN22.loc[~TRAIN22['Patient'].isin(PACIENT_val)]
# TEST = TRAIN22.loc[TRAIN22['Patient'].isin(PACIENT_val)]
# TRAIN.drop_duplicates(subset=["Patient","Weeks"],keep=False,inplace=True)
# TEST.drop_duplicates(subset=["Patient","Weeks"],keep=False,inplace=True)
# def calculate_2(row):
#     week = TEST[TEST.Patient == row["Patient"]]["Weeks"].unique().tolist()
    
#     w0 = week[0]
    
#     if row['Weeks'] == w0:
       
#         return row
# TEST23 = TEST.apply(calculate_2, axis=1)
# TEST23.dropna(inplace=True)
# print(TRAIN.shape,TEST.shape)
# TRAIN = TRAIN.append(TEST23, ignore_index = True).copy()
# print(TRAIN.shape,TEST.shape)
# print(TRAIN.shape)
# TRAIN.drop_duplicates(subset=["Patient","Weeks"],keep=False,inplace=True)
# TRAIN.reset_index(drop=True, inplace=True)   
# TEST.reset_index(drop=True, inplace=True)   
# print(TRAIN.shape)
# TEST = TRAIN22.loc[TRAIN22['Patient'].isin(PACIENT_val)]
# TEST.drop_duplicates(subset=["Patient","Weeks"],keep=False,inplace=True)
# print(TEST.shape)
TRAIN = TRAIN.append(TEST, ignore_index = True).copy()
TRAIN.drop_duplicates(subset=["Patient","Weeks"],keep=False,inplace=True)
# print(TRAIN22.shape)
# print(TEST.shape)

In [ ]:
PACIENT = TEST["Patient"].unique()
TRAIN_NEW = pd.DataFrame()
for ID in tqdm(PACIENT):
    data = TEST[TEST.Patient == ID].copy()
    data.reset_index(inplace=True,drop=True)
    data = data[:1]
    r = range(-12,134)
    count_week = len(r)
    data = data.loc[data.index.repeat(count_week)].reset_index(drop=True)
    week_predict = [i for i in r]
    data["week_predict"] = week_predict
    TRAIN_NEW = pd.concat([TRAIN_NEW, data], ignore_index=True)
TEST = TRAIN_NEW

In [ ]:
def counsruct(dataframe,test = False):
    PACIENT = dataframe["Patient"].unique()
    TRAIN_NEW = pd.DataFrame()
    for ID in tqdm(PACIENT):
        data = dataframe[dataframe.Patient == ID].copy()
        data.reset_index(inplace=True,drop=True)
        week_start, FVC_start, Percent_kt,d = data.loc[0,["Weeks","FVC","Percent","dir"]].values
        DIR=d
        DIR_DCM = DIR+ID+"/"
        d = os.listdir(DIR_DCM)
        d = sorted(os.listdir(DIR_DCM), key=lambda v:int(v.split('.')[0]))
        count_slice = len(d)
        d = {i+1:dcm for i,dcm in enumerate(d)}
        center = len(d)//2
#         arr_slice = [center-5,center-4,center-3,center-2,center-1,center,center+1,center+2,center+3,center+4,center+5]
#         arr_slice = [center-3,center-2,center-1,center,center+1,center+2,center+3]
        c=center-(center*40//100)
        arr_slice = [c]
        count_repeat = len(arr_slice)
        
        d = np.array([[d[i],int(i)] for j in range(data.shape[0]) for i in arr_slice])
        d = pd.DataFrame(d,columns=["dcm","num_slice"])
        d["num_slice"] = d["num_slice"].astype("int")
        data = data.loc[data.index.repeat(count_repeat)].reset_index(drop=True)
        data[["dcm","num_slice"]] = d[["dcm","num_slice"]]
        data["count_slice"] = count_slice
        data["week_kt"] = week_start
        data["FVC_kt"] = FVC_start
        data["Percent_kt"] = Percent_kt
        TRAIN_NEW = pd.concat([TRAIN_NEW, data], ignore_index=True)
    return TRAIN_NEW
TRAIN_C = counsruct(TRAIN)
TEST_C = counsruct(TEST,True)
# TEST_C["Weeks"] = TEST_C["week_predict"]
TEST_C["Count_weks"] = TEST_C["week_predict"]-TEST_C["week_kt"]
# TEST_C["Count_weks"] = TEST_C["Weeks"]-TEST_C["week_kt"]
TRAIN_C["Count_weks"] = TRAIN_C["Weeks"]-TRAIN_C["week_kt"]
# TRAIN_C['week_min'] = TRAIN_C['Patient'].map(TRAIN_C.groupby(['Patient']).Weeks.min())
# TRAIN_C['week_max'] = TRAIN_C['Patient'].map(TRAIN_C.groupby(['Patient']).Weeks.max())
# TEST_C['week_min'] = TEST_C['Patient'].map(TEST_C.groupby(['Patient']).week_predict.min())
# TEST_C['week_max'] = TEST_C['Patient'].map(TEST_C.groupby(['Patient']).week_predict.max())

In [ ]:
# TRAIN_C.loc[TRAIN_C.Percent_kt>94,["Percent_kt"]] = 15.2567
# TEST_C.loc[TEST_C.Percent_kt>94,["Percent_kt"]] = 15.2567

In [ ]:
r = 1
e=70
def custom_data(dataframe):
    
    dataframe["FVC_n"] = (dataframe["FVC_kt"]*100/dataframe["Percent_kt"])
    for i in range(r,e):
        name ="FVC_mean"+str(i)
        name2 = "FVC_custom"+str(i)
        dataframe[name] = (dataframe["FVC_n"]-dataframe["FVC_kt"])/(30*i)
        dataframe[name2] = (dataframe["FVC_kt"]-(dataframe["Count_weks"])*dataframe[name])-((dataframe["Count_weks"]+90))#-80
    return dataframe
TRAIN_C=custom_data(TRAIN_C)
TEST_C=custom_data(TEST_C)
name = ["FVC_custom"+str(i) for i in range(r,e)]
# TEST_C["FVC_PRE"] = TEST_C[name[5:-60]].mean(axis=1)
# TRAIN_C["FVC_PRE"] = TRAIN_C[name[5:-60]].mean(axis=1)
TEST_C["FVC_PRE"] = TEST_C[name[10:]].mean(axis=1)
TRAIN_C["FVC_PRE"] = TRAIN_C[name[10:]].mean(axis=1)
TEST_C["FVC_PRE2"] = TEST_C["FVC_PRE"]**2
TRAIN_C["FVC_PRE2"] = TRAIN_C["FVC_PRE"]**2
TEST_C["FVC_n2"] = TEST_C["FVC_n"]**2
TRAIN_C["FVC_n2"] = TRAIN_C["FVC_n"]**2
TEST_C["r1"] = TEST_C["FVC_n"] - TEST_C["FVC_PRE"]
TEST_C["r1mean"] = TEST_C[["FVC_n","FVC_PRE"]].mean(axis=1)
TEST_C["r2"] = TEST_C[["FVC_n","FVC_PRE"]].std(axis=1)
TRAIN_C["r1"] = TRAIN_C["FVC_n"] - TRAIN_C["FVC_PRE"]
TRAIN_C["r1mean"] = TRAIN_C[["FVC_n","FVC_PRE"]].mean(axis=1)
TRAIN_C["r2"] = TRAIN_C[["FVC_n","FVC_PRE"]].std(axis=1)
TRAIN_C[["Female","Male","Currently smokes","Ex-smoker","Never smoked"]]=0
TEST_C[["Female","Male","Currently smokes","Ex-smoker","Never smoked"]]=0
def calculate_height(row):
    if row['Sex'] == 'Male':
        return row['FVC_kt'] / (27.63 - 0.112 * row['Age'])
    else:
        return row['FVC_kt'] / (21.78 - 0.101 * row['Age'])
def calculate_all(row):
    if row['Sex'] == 'Male':
        row['Male'] = 1
       
    else:
        row['Female']=1
        
    if row['SmokingStatus'] == "Currently smokes":
        row['Currently smokes']= 1
    if row['SmokingStatus'] == "Ex-smoker":
        row['Ex-smoker']= 1
    if row['SmokingStatus'] == "Never smoked":
        row['Never smoked']= 1
    return row
        
TRAIN_C['Height'] = TRAIN_C.apply(calculate_height, axis=1)
TEST_C['Height'] = TEST_C.apply(calculate_height, axis=1)
TRAIN_C = TRAIN_C.apply(calculate_all, axis=1)
TEST_C = TEST_C.apply(calculate_all, axis=1)
TRAIN_C = TRAIN_C.drop(columns=['Sex', 'SmokingStatus'])
TEST_C = TEST_C.drop(columns=['Sex', 'SmokingStatus'])


In [ ]:
def construct_binned(dataframe_train,dataframe_test,column,count,combi=True):
    col = dataframe_train[column].unique().tolist()
    col.extend(dataframe_test[column].unique().tolist())
    all_col = np.array(col)
    bins = np.linspace(all_col.min(),all_col.max(),count)
    col_data_train =  dataframe_train[column].values.reshape(-1,1)
    col_data_test =  dataframe_test[column].values.reshape(-1,1)
    witch_bin = np.digitize(col_data_train,bins)
    witch_bin2 = np.digitize(col_data_test,bins)
    encoder = OneHotEncoder(sparse=False)
    encoder.fit(witch_bin)
    col_binned = encoder.transform(witch_bin)
    col_binned2 = encoder.transform(witch_bin2)
    col_binned_combi = np.hstack([col_data_train,col_binned])
    col_binned2_combi = np.hstack([col_data_test,col_binned2])
    set1 = col_binned_combi
    set2 = col_binned2_combi
    if not combi:
        set1 = col_binned
        set2 = col_binned2
    name_col_binned = [column+"_binned"+str(i) for i in range(set1.shape[1])]
    dataframe_train[name_col_binned] = set1
    dataframe_test[name_col_binned] = set2
    
    return dataframe_train, dataframe_test,name_col_binned
TRAIN_C,TEST_C,binned_age = construct_binned(TRAIN_C,TEST_C,"Age",60,True)#50#70
TRAIN_C,TEST_C,binned_height = construct_binned(TRAIN_C,TEST_C,"Height",5,True)
TRAIN_C,TEST_C,binned_Percent_kt = construct_binned(TRAIN_C,TEST_C,"Percent_kt",60,True)
TRAIN_C,TEST_C,binned_FVC_kt = construct_binned(TRAIN_C,TEST_C,"FVC_kt",60,True)#50#100
TRAIN_C,TEST_C,binned_Count_week_kt = construct_binned(TRAIN_C,TEST_C,"week_kt",15,True)
TRAIN_C,TEST_C,binned_FVC_n = construct_binned(TRAIN_C,TEST_C,"FVC_n",60,True)
TRAIN_C,TEST_C,binned_count_slice = construct_binned(TRAIN_C,TEST_C,"count_slice",15,True)

In [ ]:
Height = TRAIN_C["Height"].unique().tolist()
Height.extend(TEST_C["Height"].unique().tolist())
all_Height = np.unique(Height)
bins = np.linspace(all_Height.min(),all_Height.max(),5)
witch_bin = np.digitize(TRAIN_C.Height,bins)
witch_bin2 = np.digitize(TEST_C.Height,bins)
encoder = OneHotEncoder(sparse=False)
encoder.fit(witch_bin.reshape(-1, 1))
Height_binned = encoder.transform(witch_bin.reshape(-1, 1))
Height_binned2 = encoder.transform(witch_bin2.reshape(-1, 1))
name_height_binned = ["Height_binned"+str(i) for i in range(Height_binned.shape[1])]
TRAIN_C[name_height_binned] = Height_binned
TEST_C[name_height_binned] = Height_binned2

In [ ]:
FVC_kt_train = TRAIN_C["FVC_kt"].unique().tolist()
FVC_kt_train.extend(TEST_C["FVC_kt"].unique().tolist())
all_FVC_kt = np.unique(FVC_kt_train)
bins = np.linspace(all_FVC_kt.min(),all_FVC_kt.max(),11)
witch_bin = np.digitize(TRAIN_C.FVC_kt,bins)
witch_bin2 = np.digitize(TEST_C.FVC_kt,bins)
encoder = OneHotEncoder(sparse=False)
encoder.fit(witch_bin.reshape(-1, 1))
FVC_binned = encoder.transform(witch_bin.reshape(-1, 1))
FVC_binned2 = encoder.transform(witch_bin2.reshape(-1, 1))
# print(FVC_binned2.shape)
name_bin_fvckt = ["FVC_KT_bin"+str(i) for i in range(FVC_binned.shape[1])]
TRAIN_C[name_bin_fvckt] = FVC_binned
TEST_C[name_bin_fvckt] = FVC_binned2

In [ ]:
FVC_PRE_train = TRAIN_C["FVC_PRE"].unique().tolist()
FVC_PRE_train.extend(TEST_C["FVC_PRE"].unique().tolist())
all_FVC_PRE_kt = np.unique(FVC_PRE_train)
bins2 = np.linspace(all_FVC_PRE_kt.min(),all_FVC_PRE_kt.max(),5)
witch_bin2 = np.digitize(TRAIN_C.FVC_PRE,bins2)
witch_bin22 = np.digitize(TEST_C.FVC_PRE,bins2)
# print(witch_bin2)
encoder = OneHotEncoder(sparse=False)
encoder.fit(witch_bin.reshape(-1, 1))
FVC_PRE_binned = encoder.transform(witch_bin2.reshape(-1, 1))
FVC_PRE_binned2 = encoder.transform(witch_bin22.reshape(-1, 1))
name_bin_pre = ["FVC_PRE_bin"+str(i) for i in range(FVC_binned.shape[1])]
TRAIN_C[name_bin_pre] = FVC_PRE_binned
TEST_C[name_bin_pre] = FVC_PRE_binned2


In [ ]:
def calculate_FVC(row):
    if row['Weeks'] == row['week_kt']:
        row['FVC_PRE'] = row['FVC']
    return row
TRAIN_C = TRAIN_C.apply(calculate_FVC, axis=1)

In [ ]:
Patient = LabelEncoder()
# Sex = LabelEncoder()
# SmokingStatus = LabelEncoder()
train_pac = TRAIN_C["Patient"].unique().tolist()
train_pac.extend(TEST_C["Patient"].unique().tolist())
all_pacient = np.unique(train_pac)
Patient.fit(all_pacient)
def LE(dataframe, val=False,dense=False):
    dataframe["patiet_id"] = Patient.transform(dataframe["Patient"])
    col = ["Patient","dcm"]
    if not val:
        col.extend(["FVC","Weeks"])
    if val:
        col.extend(["week_predict"])
#         col.extend(["FVC","Weeks"])
    col.extend(name_bin_pre)
#     col.extend(name_bin_fvckt)
    col.extend(name_height_binned)
    col.extend(binned_age)
    col.extend(binned_FVC_kt)
    col.extend(binned_Percent_kt)
    col.extend(binned_FVC_n)
#     col.extend(binned_age)
    col.extend(["FVC_PRE","FVC_PRE2","count_slice","week_kt","Count_weks",'Height',"Currently smokes","Ex-smoker","Never smoked","Female","Male","r1","r2","r1mean"])
#     print(col)
    dataframe = dataframe[col]
    dataframe['dcm'] = dataframe.agg('{0[Patient]}/{0[dcm]}'.format, axis=1)
    return dataframe
TRAIN2 = LE(TRAIN_C.copy())
TEST2 = LE(TEST_C.copy(),True)


In [ ]:
train = 0
validation = 0
def Fold(dataframe):
    train = []
    val = []
    PACIENT = dataframe["Patient"].unique()
 
    for ID in PACIENT:
        
        
        d = dataframe[dataframe.Patient==ID]
        d.reset_index(inplace=True,drop=True)
        week = d["Weeks"].unique().tolist()
        if len(week)==1:
            week_train = [week[0]]
            week_val = [week[0]]
            
        else:
            week_val = week[:]
            week_train =week[:]
#         print(week_val)
        train_index = dataframe[(dataframe.Patient==ID) & (dataframe.Weeks.isin(week_train))].index.tolist()
        val_index = dataframe[(dataframe.Patient==ID) & (dataframe.Weeks.isin(week_val))].index.tolist()
        train.extend(train_index)
        val.extend(val_index)
    return train,val
train_index, val_index = Fold(TRAIN2)
# train_index2, val_index2 = Fold(TEST2)
train = TRAIN2.loc[train_index]
# validation = TEST2.loc[val_index2]
# train.reset_index(drop=True, inplace=True)    
# validation.reset_index(drop=True, inplace=True)
# train = TRAIN2.loc[train_index]
validation = TRAIN2.loc[val_index]
train.reset_index(drop=True, inplace=True)    
validation.reset_index(drop=True, inplace=True)
# train = train.sample(frac=1, random_state=123).reset_index(drop=True)
# validation = validation.sample(frac=1, random_state=123).reset_index(drop=True)

In [ ]:
 def laplace_log_likelihood(actual_fvc, predicted_fvc, confidence, return_values = False):
    """
    Calculates the modified Laplace Log Likelihood score for this competition.
    """
    sd_clipped = np.maximum(confidence, 70)
    delta = np.minimum(np.abs(actual_fvc - predicted_fvc), 1000)
    metric = - np.sqrt(2) * delta / sd_clipped - np.log(np.sqrt(2) * sd_clipped)

    if return_values:
        return metric
    else:
        return np.mean(metric)

In [ ]:
# validation = True
train_split = train.copy()
test_split = validation.copy()
X_train = train_split[train_split.columns.tolist()[3:]].copy()
X_val= test_split[test_split.columns.tolist()[3:]].copy()
X_val2= test_split[test_split.columns.tolist()[3:]].copy()
X_end = TRAIN2[train_split.columns.tolist()[3:]].copy()
X_end2 = TRAIN2[train_split.columns.tolist()[3:]].copy()
Y_end = TRAIN2["FVC"].copy()
Y_train = train_split["FVC"].copy()
Y_train2 = Y_train- train_split["FVC_PRE"]
Y_train=Y_train2.abs()
Y_val = test_split["FVC"].copy()
Y_val2 = Y_val- test_split["FVC_PRE"]
Y_val2 = Y_val2.abs()
# X_val2["Y_val"] = Y_val2


X_test = TEST2[TEST2.columns.tolist()[2:]]
X_train_kneigboards = train_split[train_split.columns.tolist()[3:]].copy()
X_val_kneigboards= test_split[test_split.columns.tolist()[3:]].copy()
X_test_kneigboards= TEST2[TEST2.columns.tolist()[2:]]

# std_scaller = StandardScaler()
# name = ["count_slice","FVC_n","Percent_kt","Count_weks","r1","r2"]
# std_scaller.fit(X_train[name])
# X_train[name] = std_scaller.transform(X_train[name])
# X_val[name] =  std_scaller.transform(X_val[name])
# val_x_std_scalled = std_scaller.transform(val_x)

alpha = 0.9
tree1 = GradientBoostingRegressor(loss='quantile', alpha=alpha,
                                n_estimators=250, max_depth=5,
                                learning_rate=.1, min_samples_leaf=49,
                                min_samples_split=49)
tree1.fit(X_train_kneigboards,Y_train)
y_upper = tree1.predict(X_val_kneigboards)


tree1.set_params(alpha=0.1)
tree1.fit(X_train_kneigboards, Y_train)
y_lower = tree1.predict(X_val_kneigboards)

tree1.set_params(loss='ls')
tree1.fit(X_train_kneigboards, Y_train)
y_pred = tree1.predict(X_val_kneigboards)
tree3 = KNeighborsRegressor(n_neighbors=252)
tree3.fit(X_train_kneigboards,Y_train)
pred_k = tree3.predict(X_val_kneigboards)

tree2 =  RandomForestRegressor(n_estimators=30,min_samples_leaf=2, min_samples_split=2)
tree2.fit(X_train,Y_train)
pred_r = tree2.predict(X_val)

tree4 = LinearRegression()
tree4.fit(X_train_kneigboards[X_train_kneigboards.columns.tolist()[:-11]],Y_train)
pred_lr = tree4.predict(X_val_kneigboards[X_val_kneigboards.columns.tolist()[:-11]])
tree5 = Ridge(alpha=0.03)
tree5.fit(X_train_kneigboards[X_train_kneigboards.columns.tolist()[:-11]],Y_train)
pred_ridge = tree5.predict(X_val_kneigboards[X_val_kneigboards.columns.tolist()[:-11]])

lf = BayesianRidge()
lf.fit(X_train_kneigboards[X_train_kneigboards.columns.tolist()[:]],Y_train)
pred_baes = lf.predict(X_val_kneigboards[X_val_kneigboards.columns.tolist()[:]])

print(X_val_kneigboards.columns.tolist()[8:-10])
print(mean_squared_error(Y_val2,y_upper, squared=False))
print(mean_squared_error(Y_val2,y_lower, squared=False))
print(mean_squared_error(Y_val2,y_pred, squared=False))
print("custom",mean_squared_error(Y_val,X_val2["FVC_PRE"], squared=False))


print("kneugboard",mean_squared_error(Y_val2,pred_k, squared=False))
print("randonf",mean_squared_error(Y_val2,pred_r, squared=False))
print("linear",mean_squared_error(Y_val2,pred_lr, squared=False))
print("ridge",mean_squared_error(Y_val2,pred_ridge, squared=False))
print("baes",mean_squared_error(Y_val2,pred_baes, squared=False))
X_val2["y_upper"] = y_upper
X_val2["y_lower"] = y_lower
X_val2["y_pred"] = y_pred
X_val2["pred_k"] = pred_k
X_val2["pred_r"] = pred_r
X_val2["pred_lr"] = pred_lr
X_val2["pred_ridge"] = pred_ridge
X_val2["baes"] = pred_baes
name = ["baes","pred_r"]
X_val2["Confidence"] = Y_val- X_val2["FVC_PRE"]
X_val2["Confidence"] = X_val2["Confidence"].abs()



# X_val2["Confidence"] = 200
# display(X_val2["Confidence"])
# X_val2["Confidence"] = (X_val2[["pred_k","FVC_PRE"]].max(axis=1) - X_val2[["pred_k","FVC_PRE"]].min(axis=1))
delta = 60
X_val2["end"] = X_val2[name].mean(axis=1)
print("----",X_val2["end"].std())
X_val2["end"]+=delta
print("mean",mean_squared_error(Y_val,X_val2["end"], squared=False))
test_split["end"]= X_val2["FVC_PRE"]
print(laplace_log_likelihood(Y_val,X_val2.pred_lr,X_val2["Confidence"]))#140
print(laplace_log_likelihood(Y_val,X_val2.FVC_PRE,X_val2.end))#140
print(laplace_log_likelihood(Y_val,X_val2.end,X_val2["Confidence"]))#140

tree1.set_params(loss='ls')
y_pred2 = tree1.predict(X_test_kneigboards)
pred_k2 = tree3.predict(X_test_kneigboards)
pred_r2 = tree2.predict(X_test)
pred_lr2 = tree4.predict(X_test_kneigboards[X_test_kneigboards.columns.tolist()[:-11]])
pred_ridge2 = tree5.predict(X_test_kneigboards[X_test_kneigboards.columns.tolist()[:-11]])
pred_baes2 = lf.predict(X_test_kneigboards[X_test_kneigboards.columns.tolist()[:]])
TEST2["y_pred"] = y_pred2
TEST2["pred_k"] = pred_k2
TEST2["pred_r"] = pred_r2
TEST2["pred_lr"] = pred_lr2
TEST2["pred_ridge"] = pred_ridge2
TEST2["baes"] = pred_baes2
TEST2["end"] = TEST2[name].mean(axis=1)
TEST2["end"]+=delta
# TEST2["Confidence"] =  170+(TEST2["Count_weks"])
TEST2['Patient_Week'] = TEST2.agg('{0[Patient]}_{0[week_predict]}'.format, axis=1)
SUBMISSINO1_pred2 = TEST2[['Patient_Week','FVC_PRE','end',]]
SUBMISSINO1_pred2.columns = ["Patient_Week","FVC","Confidence"]
SUBMISSINO1_pred2.to_csv("submission.csv", index=False)
SUBMISSINO1_pred2
# # display(SUBMISSINO1_pred2)
# # # plt.figure(figsize=(15,8))
# # # sns.barplot(y=X_val_kneigboards.columns, x=tree4.feature_importances_, )
# # # plt.figure(figsize=(15,8))
# # # sns.barplot(y=X_val.columns, x=tree2.feature_importances_, )
# train_patients = test_split.loc[test_split.Percent_kt>90]['Patient'].unique()
# fig, ax = plt.subplots(40, 1, figsize=(20, 70))

# for i in range(0,40):
#     patient_log = test_split[test_split['Patient'] == train_patients[i]]
#     ax[i].set_title(patient_log[["Percent_kt","FVC_n"]].values[0])
# #     print(patient_log["Percent_kt"])
#     ax[i].plot(patient_log['Weeks'], patient_log['FVC'], label='truth')
#     ax[i].plot(patient_log['Weeks'], patient_log['end'], label='prediction')
#     ax[i].legend()
# #     -7.060406176501061

In [ ]:
# ['FVC_KT_bin7', 'FVC_KT_bin8', 'Height_binned0', 'Height_binned1', 'Height_binned2', 'Height_binned3', 'Age', 'count_slice', 'week_kt', 'Count_weks', 'Height', 'FVC_kt', 'Currently smokes', 'Ex-smoker']
# 112.12877222068204
# 158.5906741062012
# 60.857885911243045
# custom 218.45109162754719
# kneugboard 137.54239529961768
# randonf 135.94007416439462
# linear 151.7000903643887
# ridge 151.70009960419222
# baes 146.3852677292098
# mean 2674.0087667436064
# -18.391432439384857
# -6.5973666610214705
# -18.390146002170916